<a href="https://colab.research.google.com/github/Minh-A/Pytorch_DeepLearning/blob/main/LSTM_pytorch_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [4]:
# 구글드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015) ###### <- 이거 뭐냐?(랜덤시드 같은 것)

# define 'device' to upload tensor in gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#시간을 계산하는 함수
import time, datetime
def clock(start):
    sec = time.time() - start #현재시간 - 시스템초기시간
    times = str(datetime.timedelta(seconds = sec)).split(".") # 시간:분:초로 변환
    times = times[0]
    return times

# Data Load

In [6]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_pytorch(Dacon)/input/train.csv", encoding = 'euc-kr')
train['DateTime'] = pd.to_datetime(train.DateTime)  # 데이터 타입을 object에서 datetime으로
train['date'] = train.DateTime.dt.date  # 날짜만 뽑음
train  = train.groupby('date').sum().reset_index()  # 날짜별로 묶음

In [7]:
train

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,281,266,73,1826
1,2018-09-10,264,247,51,2092
2,2018-09-11,329,310,58,1998
3,2018-09-12,300,287,45,2595
4,2018-09-13,378,344,50,3845
...,...,...,...,...,...
787,2020-11-04,4516,4472,1196,112683
788,2020-11-05,4155,4037,1044,102901
789,2020-11-06,3663,3576,825,88015
790,2020-11-07,2472,2417,531,57386


In [56]:
train.describe()

,사용자,세션,신규방문자,페이지뷰
count,792.000000,792.000000,792.000000,792.000000
mean,1095.214646,1086.728535,268.070707,24236.194444
std,979.730110,960.880328,241.548288,28493.828455
min,1.000000,1.000000,1.000000,2.000000
25%,292.000000,294.750000,84.750000,2552.250000
50%,625.500000,649.000000,194.500000,8416.000000
75%,1739.250000,1696.500000,363.750000,44152.000000
max,5418.000000,5400.000000,1691.000000,155435.000000


In [81]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_pytorch(Dacon)/input/train.csv", encoding = 'euc-kr')

In [82]:
train

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2018-09-09 00:00:00,19,19,8,206
1,2018-09-09 01:00:00,20,19,9,259
2,2018-09-09 02:00:00,12,9,1,48
3,2018-09-09 03:00:00,10,10,2,102
4,2018-09-09 04:00:00,6,5,3,18
...,...,...,...,...,...
19003,2020-11-08 19:00:00,124,123,19,3128
19004,2020-11-08 20:00:00,166,159,29,4864
19005,2020-11-08 21:00:00,184,173,32,3426
19006,2020-11-08 22:00:00,163,155,34,2845


# Data Preprocessing

In [89]:
train.shape

(792, 5)

In [ ]:
from sklearn.preprocessing import StandardScaler

#iloc만 해도 데이터 셋 자체가 바뀌어짐
# 표준화
train.iloc[:,1:]
scaler = StandardScaler()
scaler.fit(train.iloc[:,1:])
train.iloc[:,1:] = scaler.transform(train.iloc[:,1:])
train.iloc[:,1:].shape

(792, 4)

In [90]:
from sklearn.preprocessing import MinMaxScaler

# 정규화
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()


train.iloc[:,1:]
transformer = MinMaxScaler()
transformer.fit(train.iloc[:,1:])
train.iloc[:,1:] = transformer.transform(train.iloc[:,1:])

In [109]:
train

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,281,266,73,1826
1,2018-09-10,264,247,51,2092
2,2018-09-11,329,310,58,1998
3,2018-09-12,300,287,45,2595
4,2018-09-13,378,344,50,3845
...,...,...,...,...,...
787,2020-11-04,4516,4472,1196,112683
788,2020-11-05,4155,4037,1044,102901
789,2020-11-06,3663,3576,825,88015
790,2020-11-07,2472,2417,531,57386


In [37]:
# 전체 텐서 만들기
input_window = 24
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: ].values

In [110]:
print(window_x.shape)
print(window_y.shape)

torch.Size([755, 30, 4])
torch.Size([755, 7, 4])


In [57]:
input_window = 30
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: ].values

In [ ]:
train

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,-0.831585,-0.854682,-0.808095,-0.786990
1,2018-09-10,-0.848948,-0.874468,-0.899232,-0.777649
2,2018-09-11,-0.782561,-0.808862,-0.870234,-0.780950
3,2018-09-12,-0.812180,-0.832813,-0.924087,-0.759985
4,2018-09-13,-0.732516,-0.773455,-0.903374,-0.716088
...,...,...,...,...,...
787,2020-11-04,3.493765,3.525320,3.844017,3.106030
788,2020-11-05,3.125064,3.072324,3.214345,2.762511
789,2020-11-06,2.622567,2.592253,2.307121,2.239752
790,2020-11-07,1.406158,1.385305,1.089204,1.164138


In [111]:
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()

mini
size

사용자        5417
세션         5399
신규방문자      1690
페이지뷰     155433
dtype: int64

In [112]:
print(mini)
print(size)

사용자      1
세션       1
신규방문자    1
페이지뷰     2
dtype: int64
사용자        5417
세션         5399
신규방문자      1690
페이지뷰     155433
dtype: int64


In [104]:
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()
train.iloc[:,1:] = (train.iloc[:,1:] -  mini) / size
train

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,0.051689,0.049083,0.042604,0.011735
1,2018-09-10,0.048551,0.045564,0.029586,0.013446
2,2018-09-11,0.060550,0.057233,0.033728,0.012842
3,2018-09-12,0.055197,0.052973,0.026036,0.016682
4,2018-09-13,0.069596,0.063530,0.028994,0.024724
...,...,...,...,...,...
787,2020-11-04,0.833487,0.828116,0.707101,0.724949
788,2020-11-05,0.766845,0.747546,0.617160,0.662015
789,2020-11-06,0.676020,0.662160,0.487574,0.566244
790,2020-11-07,0.456157,0.447490,0.313609,0.369188


In [8]:
# 한번만!!!!!!!!!!!! 계속하면 0이됨
# scaling
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()
train.iloc[:,1:] = (train.iloc[:,1:] -  mini) / size

input_window = 30
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: ].values

# Model

In [9]:
# Model 생성
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        self.hidden_lstm = nn.LSTM(input_size = hidden_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        
        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4)

  
model = LSTM(input_size = 4, hidden_size = 30).to(device)

In [11]:
model

LSTM(
  (lstm): LSTM(4, 30, batch_first=True)
  (hidden_lstm): LSTM(30, 30, batch_first=True)
  (time_fc): Linear(in_features=30, out_features=4, bias=True)
)

In [ ]:
# Model 생성33333333333333333333333333333333333333333
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):                 #사용할 레이어를 정의함, 모델의 부품을 생성(__init__)
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size = input_size,   #인풋
                            hidden_size = 64,
                            batch_first=True)
        
        self.hidden_lstm = nn.LSTM(input_size = 64,    
                                   hidden_size = 32,
                                   batch_first=True)
        
        self.hidden_lstm4 = nn.LSTM(input_size = 32,
                                   hidden_size = 64,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)   #아웃풋

    
    def forward(self, x_time):                                    #위에 정의한 레이어들을 적절하게 배치하여 모델을 완성, 실질적으로 모델 생성(forward)
    
        out_time, _ = self.lstm(x_time) #인풋

        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm4(out_time)

        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4) #아웃풋

  
model = LSTM(input_size = 4, hidden_size = 64).to(device)

In [ ]:
# 이거는 사용할 레이어들만 나옴, 모델의 구성부품만 나옴
model

LSTM(
  (lstm): LSTM(4, 64, batch_first=True)
  (hidden_lstm): LSTM(64, 32, batch_first=True)
  (hidden_lstm4): LSTM(32, 64, batch_first=True)
  (time_fc): Linear(in_features=64, out_features=4, bias=True)
)

* https://sanghyu.tistory.com/52

In [198]:
# Model 생성4444444444444444444444444444444444444
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.01, bidirectional=True):                 #사용할 레이어를 정의함, 모델의 부품을 생성(__init__)
        super(GRU, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.GRU(input_size = input_size,   #인풋
                            hidden_size = hidden_size,
                            batch_first=True)
        
        self.hidden_lstm = nn.GRU(input_size = 30,    
                                   hidden_size = 30,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)   #아웃풋
        self.fn = nn.Tanh()

    
    def forward(self, x_time):                                    #위에 정의한 레이어들을 적절하게 배치하여 모델을 완성, 실질적으로 모델 생성(forward)
    
        out_time, _ = self.lstm(x_time) #인풋

        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)

        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4) #아웃풋

  
model = GRU(input_size = 4, hidden_size = 30).to(device)

In [199]:
model

GRU(
  (lstm): GRU(4, 30, batch_first=True)
  (hidden_lstm): GRU(30, 30, batch_first=True)
  (time_fc): Linear(in_features=30, out_features=4, bias=True)
  (fn): Tanh()
)

In [ ]:
# Model 생성2222222222222222222222222222222222222
# 결과는 4.7임 원래 코드는 4.3 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        self.hidden_lstm = nn.LSTM(input_size = hidden_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        
        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4)
    
model = LSTM(input_size = 4, hidden_size = 64).to(device)

In [ ]:
print(model)

LSTM(
  (lstm): LSTM(4, 64, batch_first=True)
  (hidden_lstm): LSTM(64, 64, batch_first=True)
  (time_fc): Linear(in_features=64, out_features=4, bias=True)
)


In [60]:
print(window_x.shape)
print(window_y.shape)

(755, 30, 4)
(755, 7, 4)


모델학습

In [114]:
window_x.shape

torch.Size([755, 30, 4])

In [121]:
window_x

array([[[0.05168913, 0.04908316, 0.04260355, 0.01173496],
        [0.04855086, 0.04556399, 0.0295858 , 0.01344631],
        [0.06055012, 0.05723282, 0.03372781, 0.01284155],
        ...,
        [0.01421451, 0.01370624, 0.01242604, 0.002181  ],
        [0.0182758 , 0.01778107, 0.03017751, 0.00308171],
        [0.01255307, 0.01240971, 0.01360947, 0.00156337]],

       [[0.04855086, 0.04556399, 0.0295858 , 0.01344631],
        [0.06055012, 0.05723282, 0.03372781, 0.01284155],
        [0.0551966 , 0.05297277, 0.0260355 , 0.01668243],
        ...,
        [0.0182758 , 0.01778107, 0.03017751, 0.00308171],
        [0.01255307, 0.01240971, 0.01360947, 0.00156337],
        [0.00978401, 0.00981663, 0.01301775, 0.00137037]],

       [[0.06055012, 0.05723282, 0.03372781, 0.01284155],
        [0.0551966 , 0.05297277, 0.0260355 , 0.01668243],
        [0.06959572, 0.06353028, 0.02899408, 0.02472448],
        ...,
        [0.01255307, 0.01240971, 0.01360947, 0.00156337],
        [0.00978401, 0.009816

* https://sacko.tistory.com/38

In [12]:
# Model학습
window_x = torch.tensor(window_x).float().to(device)
window_y = torch.tensor(window_y).float().to(device)

# Train model
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.MSELoss(size_average = True)
num_epochs  = 700
train_error = []

for t in range(num_epochs):
    train_pred = model(window_x)
    loss = criterion(train_pred, window_y) ### trend
    train_error.append(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if t % 10 == 0 and t !=0:
        print(f"{t} Epochs train MSE: {loss.item():1.5f}")

10 Epochs train MSE: 0.04152
20 Epochs train MSE: 0.03147
30 Epochs train MSE: 0.02892
40 Epochs train MSE: 0.02799
50 Epochs train MSE: 0.02527
60 Epochs train MSE: 0.01433
70 Epochs train MSE: 0.01096
80 Epochs train MSE: 0.00983
90 Epochs train MSE: 0.00898
100 Epochs train MSE: 0.00867
110 Epochs train MSE: 0.00842
120 Epochs train MSE: 0.00830
130 Epochs train MSE: 0.00822
140 Epochs train MSE: 0.00818
150 Epochs train MSE: 0.00815
160 Epochs train MSE: 0.00812
170 Epochs train MSE: 0.00810
180 Epochs train MSE: 0.00809
190 Epochs train MSE: 0.00807
200 Epochs train MSE: 0.00806
210 Epochs train MSE: 0.00804
220 Epochs train MSE: 0.00803
230 Epochs train MSE: 0.00801
240 Epochs train MSE: 0.00800
250 Epochs train MSE: 0.00798
260 Epochs train MSE: 0.00796
270 Epochs train MSE: 0.00794
280 Epochs train MSE: 0.00792
290 Epochs train MSE: 0.00790
300 Epochs train MSE: 0.00788
310 Epochs train MSE: 0.00786
320 Epochs train MSE: 0.00783
330 Epochs train MSE: 0.00781
340 Epochs train MS

# Prediction & CSV 

In [51]:
train_pred

tensor([[[43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131],
         ...,
         [43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131]],

        [[43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131],
         ...,
         [43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131]],

        [[43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131],
         ...,
         [43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131]],

        ...,

        [[43.9217, 43.4527, 11.1772, 86.2131],
         [43.9217, 43.4527, 11.1772, 86.2131]

In [ ]:
train_pred.shape

torch.Size([755, 7, 4])

In [ ]:
last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device)
last_month

tensor([[[0.2719, 0.2669, 0.1604, 0.2021],
         [0.2902, 0.2849, 0.1710, 0.2242],
         [0.3109, 0.3047, 0.2118, 0.2249],
         [0.5704, 0.5557, 0.4414, 0.4659],
         [0.5141, 0.5053, 0.3686, 0.4430],
         [0.5595, 0.5477, 0.4509, 0.4565],
         [0.4957, 0.4869, 0.3331, 0.3698],
         [0.3836, 0.3669, 0.1947, 0.3030],
         [0.3415, 0.3404, 0.2308, 0.2975],
         [0.3063, 0.2978, 0.1964, 0.2278],
         [0.4685, 0.4599, 0.3550, 0.3588],
         [0.5034, 0.5016, 0.4237, 0.3637],
         [0.4473, 0.4466, 0.3515, 0.3213],
         [0.3982, 0.3958, 0.3325, 0.2804],
         [0.3867, 0.3769, 0.2805, 0.2870],
         [0.3005, 0.2969, 0.2751, 0.1814],
         [0.3762, 0.3712, 0.3639, 0.2516],
         [0.4886, 0.4849, 0.3923, 0.3579],
         [0.5981, 0.6036, 0.6379, 0.5542],
         [0.4883, 0.4845, 0.4178, 0.4242],
         [0.5132, 0.5131, 0.4627, 0.4058],
         [0.4320, 0.4223, 0.3320, 0.3416],
         [0.3345, 0.3289, 0.2491, 0.2403],
         [0

In [ ]:
last_month.shape

torch.Size([1, 30, 4])

In [123]:
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_pytorch(Dacon)/input/submission.csv", encoding = 'euc-kr')
last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device)

In [124]:
submission

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,0,0,0,0
1,2020-11-10,0,0,0,0
2,2020-11-11,0,0,0,0
3,2020-11-12,0,0,0,0
4,2020-11-13,0,0,0,0
...,...,...,...,...,...
56,2021-01-04,0,0,0,0
57,2021-01-05,0,0,0,0
58,2021-01-06,0,0,0,0
59,2021-01-07,0,0,0,0


In [ ]:
model(last_month)

tensor([[[0.3109, 0.3075, 0.2104, 0.2576],
         [0.8317, 0.8310, 0.7240, 0.7188],
         [0.8340, 0.8556, 0.7645, 0.7274],
         [0.8019, 0.8213, 0.6864, 0.6791],
         [0.8205, 0.8413, 0.6917, 0.7049],
         [0.7203, 0.7382, 0.5990, 0.6443],
         [0.3676, 0.3605, 0.2497, 0.3002]]], device='cuda:0',
       grad_fn=<ViewBackward>)

In [ ]:
next_week = model(last_month)
last_month = torch.cat([last_month[-7:], next_week], axis = 1)

In [46]:
last_month

tensor([[[1.7700e+02, 1.6600e+02, 4.0000e+01, 4.0460e+03],
         [1.4900e+02, 1.3800e+02, 2.1000e+01, 3.3930e+03],
         [1.2800e+02, 1.2900e+02, 3.2000e+01, 3.5970e+03],
         [1.2100e+02, 1.1800e+02, 3.0000e+01, 2.9140e+03],
         [1.4500e+02, 1.3800e+02, 2.7000e+01, 2.7160e+03],
         [1.6300e+02, 1.6200e+02, 2.7000e+01, 2.4030e+03],
         [1.2900e+02, 1.1900e+02, 2.7000e+01, 2.9460e+03],
         [1.5000e+02, 1.4900e+02, 3.1000e+01, 3.3650e+03],
         [1.2900e+02, 1.3300e+02, 2.2000e+01, 2.5710e+03],
         [9.1000e+01, 8.3000e+01, 1.6000e+01, 1.8230e+03],
         [6.2000e+01, 5.5000e+01, 1.2000e+01, 1.2710e+03],
         [3.0000e+01, 2.8000e+01, 8.0000e+00, 3.4500e+02],
         [1.7000e+01, 1.8000e+01, 1.0000e+00, 1.4800e+02],
         [1.9000e+01, 2.0000e+01, 5.0000e+00, 2.7500e+02],
         [1.7000e+01, 1.6000e+01, 7.0000e+00, 1.7000e+02],
         [1.6000e+01, 1.5000e+01, 3.0000e+00, 1.5700e+02],
         [2.2000e+01, 2.5000e+01, 7.0000e+00, 2.5200e+02

In [50]:
pred_week = next_week.cpu().detach().numpy().reshape(output_window,4)
pred_week = pred_week * size.values + mini.values
pred_week = pred_week.astype(int)

print(pred_week)

NameError: ignored

In [ ]:
pred_week.shape

(7, 4)

In [ ]:
(len(submission) - output_window)//7 + 1

8

In [129]:
len(submission)//7

8

In [ ]:
# output_window는 일주일인 7일을 의미
output_window

7

In [126]:
(len(submission) - output_window)//7 + 2

9

In [140]:
last_month.shape

torch.Size([1, 30, 4])

In [139]:
last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device)
last_month

tensor([[[0.2719, 0.2669, 0.1604, 0.2021],
         [0.2902, 0.2849, 0.1710, 0.2242],
         [0.3109, 0.3047, 0.2118, 0.2249],
         [0.5704, 0.5557, 0.4414, 0.4659],
         [0.5141, 0.5053, 0.3686, 0.4430],
         [0.5595, 0.5477, 0.4509, 0.4565],
         [0.4957, 0.4869, 0.3331, 0.3698],
         [0.3836, 0.3669, 0.1947, 0.3030],
         [0.3415, 0.3404, 0.2308, 0.2975],
         [0.3063, 0.2978, 0.1964, 0.2278],
         [0.4685, 0.4599, 0.3550, 0.3588],
         [0.5034, 0.5016, 0.4237, 0.3637],
         [0.4473, 0.4466, 0.3515, 0.3213],
         [0.3982, 0.3958, 0.3325, 0.2804],
         [0.3867, 0.3769, 0.2805, 0.2870],
         [0.3005, 0.2969, 0.2751, 0.1814],
         [0.3762, 0.3712, 0.3639, 0.2516],
         [0.4886, 0.4849, 0.3923, 0.3579],
         [0.5981, 0.6036, 0.6379, 0.5542],
         [0.4883, 0.4845, 0.4178, 0.4242],
         [0.5132, 0.5131, 0.4627, 0.4058],
         [0.4320, 0.4223, 0.3320, 0.3416],
         [0.3345, 0.3289, 0.2491, 0.2403],
         [0

In [160]:
last_month = torch.cat([last_month[-7:], next_week], axis = 1)
last_month

tensor([[[0.3629, 0.3644, 0.2732, 0.3200],
         [0.7821, 0.7970, 0.6833, 0.7038],
         [0.8446, 0.8400, 0.7563, 0.7715],
         [0.8275, 0.8262, 0.7524, 0.7531],
         [0.8854, 0.8948, 0.8303, 0.8005],
         [0.7381, 0.7399, 0.6661, 0.6361],
         [0.2763, 0.2613, 0.2123, 0.2185],
         [0.2798, 0.2842, 0.2000, 0.2583],
         [0.8492, 0.8798, 0.7275, 0.7673],
         [0.5630, 0.5577, 0.4222, 0.4592],
         [0.4542, 0.4542, 0.3271, 0.3638],
         [0.4400, 0.4412, 0.3266, 0.3600],
         [0.3790, 0.3750, 0.2696, 0.3062],
         [0.2932, 0.2870, 0.1911, 0.2293],
         [0.3243, 0.3246, 0.2110, 0.2584],
         [0.5550, 0.5592, 0.4398, 0.4792],
         [0.4681, 0.4658, 0.3506, 0.3788],
         [0.4159, 0.4135, 0.3005, 0.3303],
         [0.3720, 0.3687, 0.2568, 0.2931],
         [0.2940, 0.2897, 0.1821, 0.2242],
         [0.2757, 0.2763, 0.1641, 0.2043],
         [0.3125, 0.3151, 0.1988, 0.2288],
         [0.4687, 0.4707, 0.3672, 0.3770],
         [0

In [155]:
last_month[0:]

tensor([[[0.2719, 0.2669, 0.1604, 0.2021],
         [0.2902, 0.2849, 0.1710, 0.2242],
         [0.3109, 0.3047, 0.2118, 0.2249],
         [0.5704, 0.5557, 0.4414, 0.4659],
         [0.5141, 0.5053, 0.3686, 0.4430],
         [0.5595, 0.5477, 0.4509, 0.4565],
         [0.4957, 0.4869, 0.3331, 0.3698],
         [0.3836, 0.3669, 0.1947, 0.3030],
         [0.3415, 0.3404, 0.2308, 0.2975],
         [0.3063, 0.2978, 0.1964, 0.2278],
         [0.4685, 0.4599, 0.3550, 0.3588],
         [0.5034, 0.5016, 0.4237, 0.3637],
         [0.4473, 0.4466, 0.3515, 0.3213],
         [0.3982, 0.3958, 0.3325, 0.2804],
         [0.3867, 0.3769, 0.2805, 0.2870],
         [0.3005, 0.2969, 0.2751, 0.1814],
         [0.3762, 0.3712, 0.3639, 0.2516],
         [0.4886, 0.4849, 0.3923, 0.3579],
         [0.5981, 0.6036, 0.6379, 0.5542],
         [0.4883, 0.4845, 0.4178, 0.4242],
         [0.5132, 0.5131, 0.4627, 0.4058],
         [0.4320, 0.4223, 0.3320, 0.3416],
         [0.3345, 0.3289, 0.2491, 0.2403],
         [0

In [161]:
last_month[:,7:,:]

tensor([[[0.2798, 0.2842, 0.2000, 0.2583],
         [0.8492, 0.8798, 0.7275, 0.7673],
         [0.5630, 0.5577, 0.4222, 0.4592],
         [0.4542, 0.4542, 0.3271, 0.3638],
         [0.4400, 0.4412, 0.3266, 0.3600],
         [0.3790, 0.3750, 0.2696, 0.3062],
         [0.2932, 0.2870, 0.1911, 0.2293],
         [0.3243, 0.3246, 0.2110, 0.2584],
         [0.5550, 0.5592, 0.4398, 0.4792],
         [0.4681, 0.4658, 0.3506, 0.3788],
         [0.4159, 0.4135, 0.3005, 0.3303],
         [0.3720, 0.3687, 0.2568, 0.2931],
         [0.2940, 0.2897, 0.1821, 0.2242],
         [0.2757, 0.2763, 0.1641, 0.2043],
         [0.3125, 0.3151, 0.1988, 0.2288],
         [0.4687, 0.4707, 0.3672, 0.3770],
         [0.5105, 0.5105, 0.4232, 0.4160],
         [0.4923, 0.4895, 0.4032, 0.3986],
         [0.4334, 0.4234, 0.3462, 0.3445],
         [0.3311, 0.3184, 0.2456, 0.2464],
         [0.3128, 0.3125, 0.2227, 0.2325],
         [0.4754, 0.4866, 0.3821, 0.3936],
         [0.6386, 0.6530, 0.5639, 0.5606],
         [0

In [171]:
train.iloc[-30:,1:]

,사용자,세션,신규방문자,페이지뷰
762,0.341517,0.340433,0.230769,0.297459
763,0.306258,0.297833,0.196450,0.227847
764,0.468525,0.459900,0.355030,0.358836
765,0.503415,0.501574,0.423669,0.363668
766,0.447296,0.446564,0.351479,0.321251
767,0.398191,0.395814,0.332544,0.280404
768,0.386745,0.376922,0.280473,0.286966
769,0.300535,0.296907,0.275148,0.181422
770,0.376223,0.371180,0.363905,0.251613
771,0.488647,0.484905,0.392308,0.357858


In [172]:
train

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,0.051689,0.049083,0.042604,0.011735
1,2018-09-10,0.048551,0.045564,0.029586,0.013446
2,2018-09-11,0.060550,0.057233,0.033728,0.012842
3,2018-09-12,0.055197,0.052973,0.026036,0.016682
4,2018-09-13,0.069596,0.063530,0.028994,0.024724
...,...,...,...,...,...
787,2020-11-04,0.833487,0.828116,0.707101,0.724949
788,2020-11-05,0.766845,0.747546,0.617160,0.662015
789,2020-11-06,0.676020,0.662160,0.487574,0.566244
790,2020-11-07,0.456157,0.447490,0.313609,0.369188


In [178]:
print("\n", "dl")


 dl


In [180]:
size.values

array([  5417,   5399,   1690, 155433])

In [15]:
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_pytorch(Dacon)/input/submission.csv", encoding = 'euc-kr')

#last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device) <- 수정 전
last_month = train.iloc[-30:,1:].values[np.newaxis,...] # <- 수정 후
last_month = torch.tensor(last_month).float().to(device) # <- 수정 후

for start in range((len(submission) - output_window)//7 + 2):
    start = start * 7
    print("\n", start, "\n")
    
    next_week = model(last_month)
    print(next_week, "아이후야 예측")

    print(last_month[:,7:,:], "아이고 합치기")

    #last_month = torch.cat([last_month[-7:], next_week], axis = 1) <- 수정 전
    last_month = torch.cat([last_month[:,7:,:], next_week], axis = 1)# <- 수정 후

    pred_week = next_week.cpu().detach().numpy().reshape(output_window,4)

    print(pred_week,"1")

    pred_week = pred_week * size.values + mini.values
    print(pred_week,"2")
    pred_week = pred_week.astype(int)
    print(pred_week,"3")

    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
        print(submission.iloc[start :, 1:].shape[0])
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
        print(submission.iloc[start : start + output_window, 1:])
submission


 0 

tensor([[[0.4957, 0.5085, 0.3844, 0.4302],
         [0.5938, 0.6083, 0.5013, 0.5265],
         [0.5158, 0.5209, 0.4269, 0.4463],
         [0.3549, 0.3560, 0.2717, 0.2904],
         [0.2513, 0.2383, 0.1698, 0.1680],
         [0.2337, 0.2035, 0.1332, 0.1282],
         [0.2955, 0.2676, 0.1791, 0.1916]]], device='cuda:0',
       grad_fn=<ViewBackward>) 아이후야 예측
tensor([[[0.3005, 0.2969, 0.2751, 0.1814],
         [0.3762, 0.3712, 0.3639, 0.2516],
         [0.4886, 0.4849, 0.3923, 0.3579],
         [0.5981, 0.6036, 0.6379, 0.5542],
         [0.4883, 0.4845, 0.4178, 0.4242],
         [0.5132, 0.5131, 0.4627, 0.4058],
         [0.4320, 0.4223, 0.3320, 0.3416],
         [0.3345, 0.3289, 0.2491, 0.2403],
         [0.2994, 0.2969, 0.1882, 0.2504],
         [0.5942, 0.5755, 0.4574, 0.5109],
         [0.6149, 0.6040, 0.4828, 0.4987],
         [0.6299, 0.6184, 0.4787, 0.5629],
         [0.7297, 0.7307, 0.5450, 0.6938],
         [0.5979, 0.5840, 0.4231, 0.5801],
         [0.3701, 0.3745, 0.2639,

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,2686,2746,650,66874
1,2020-11-10,3217,3285,848,81832
2,2020-11-11,2795,2813,722,69366
3,2020-11-12,1923,1922,460,45137
4,2020-11-13,1362,1287,287,26111
...,...,...,...,...,...
56,2021-01-04,2280,2186,544,52778
57,2021-01-05,1893,1803,428,41187
58,2021-01-06,1895,1798,416,40527
59,2021-01-07,2355,2274,546,54143


In [14]:
# 예측 및 제출파일 생성
# submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_pytorch(Dacon)/input/submission.csv", encoding = 'euc-kr')
# last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device)
for start in range((len(submission) - output_window)//7 + 2):
    start = start * 7
    next_week = model(last_month)
    last_month = torch.cat([last_month[-7:], next_week], axis = 1)
    
    pred_week = next_week.cpu().detach().numpy().reshape(output_window,4)
    pred_week = pred_week * size.values + mini.values
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
submission

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,3274,3210,829,79114
1,2020-11-10,2970,2873,735,70120
2,2020-11-11,2334,2259,556,53380
3,2020-11-12,1847,1802,423,39671
4,2020-11-13,1866,1827,414,39793
...,...,...,...,...,...
56,2021-01-04,2256,2285,559,57347
57,2021-01-05,2103,2176,522,53609
58,2021-01-06,2517,2619,645,65851
59,2021-01-07,3127,3240,833,82795


In [16]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/LSTM_pytorch(Dacon)/input/제출1_GRU6.csv', index = False, encoding = 'euc-kr')

In [44]:
last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device)

In [45]:
last_month

tensor([[[1.7700e+02, 1.6600e+02, 4.0000e+01, 4.0460e+03],
         [1.4900e+02, 1.3800e+02, 2.1000e+01, 3.3930e+03],
         [1.2800e+02, 1.2900e+02, 3.2000e+01, 3.5970e+03],
         [1.2100e+02, 1.1800e+02, 3.0000e+01, 2.9140e+03],
         [1.4500e+02, 1.3800e+02, 2.7000e+01, 2.7160e+03],
         [1.6300e+02, 1.6200e+02, 2.7000e+01, 2.4030e+03],
         [1.2900e+02, 1.1900e+02, 2.7000e+01, 2.9460e+03],
         [1.5000e+02, 1.4900e+02, 3.1000e+01, 3.3650e+03],
         [1.2900e+02, 1.3300e+02, 2.2000e+01, 2.5710e+03],
         [9.1000e+01, 8.3000e+01, 1.6000e+01, 1.8230e+03],
         [6.2000e+01, 5.5000e+01, 1.2000e+01, 1.2710e+03],
         [3.0000e+01, 2.8000e+01, 8.0000e+00, 3.4500e+02],
         [1.7000e+01, 1.8000e+01, 1.0000e+00, 1.4800e+02],
         [1.9000e+01, 2.0000e+01, 5.0000e+00, 2.7500e+02],
         [1.7000e+01, 1.6000e+01, 7.0000e+00, 1.7000e+02],
         [1.6000e+01, 1.5000e+01, 3.0000e+00, 1.5700e+02],
         [2.2000e+01, 2.5000e+01, 7.0000e+00, 2.5200e+02